In [1]:
import json

In [ ]:
with open('data/train/_annotations.coco.json') as f:
    data = json.load(f)
print(data.keys())

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])


In [5]:
images = data['images']
labels = data['annotations']
len(images) , len(labels)

(2571, 4044)

In [8]:
data['categories']

[{'id': 0, 'name': 'Recycling-Waste', 'supercategory': 'none'},
 {'id': 1, 'name': 'cardboard', 'supercategory': 'Recycling-Waste'},
 {'id': 2, 'name': 'glass', 'supercategory': 'Recycling-Waste'},
 {'id': 3, 'name': 'metal', 'supercategory': 'Recycling-Waste'},
 {'id': 4, 'name': 'paper', 'supercategory': 'Recycling-Waste'},
 {'id': 5, 'name': 'plastic', 'supercategory': 'Recycling-Waste'}]

In [11]:
images[:1]

[{'id': 0,
  'license': 1,
  'file_name': 'metal152_jpg.rf.06414376028f03cc3a360e50505abb0f.jpg',
  'height': 384,
  'width': 512,
  'date_captured': '2025-11-30T15:32:11+00:00',
  'extra': {'name': 'metal152.jpg'}}]